In [1]:
import pandas as pd
from io import StringIO
import altair as alt

pd.set_option('display.max_columns', None)

In [2]:
def read_cdc_wonder(filepath):
    """
    Read CDC WONDER data file into a pandas DataFrame.
    
    Parameters:
    filepath (str): Path to the CDC WONDER text file
    
    Returns:
    pandas.DataFrame: Processed data from the file
    """
    # Try different encodings
    encodings = ['windows-1252', 'cp1252', 'latin1', 'iso-8859-1']
    
    for encoding in encodings:
        try:
            # Read the file content
            with open(filepath, 'r', encoding=encoding) as file:
                lines = file.readlines()
            break  # If successful, break the loop
        except UnicodeDecodeError:
            if encoding == encodings[-1]:  # If this was the last encoding to try
                raise
            continue
    
    # Find where the metadata section begins
    data_end_idx = next(i for i, line in enumerate(lines) if line.strip() == '"---"')
    print(f'Data ends on line {data_end_idx} out of {len(lines)} lines')
    
    # Join only the data portion of the file
    data_content = ''.join(lines[:data_end_idx])
    
    # Read the CSV content into a DataFrame
    df = pd.read_csv(
        StringIO(data_content),
        # delim_whitespace=True,
        sep="\t",
        quotechar='"',
        # skipinitialspace=True,
        na_values=['Not Applicable', 'Unreliable'],
        thousands=','
    )
    
    # Clean up column names
    df.columns = df.columns.str.strip()
    # df.columns = df.columns.str.lower()
    
    
    # Convert numeric columns to appropriate types
    numeric_columns = ['Deaths', 'Population', 'Crude Rate', 
                      'Crude Rate Lower 95% Confidence Interval',
                      'Crude Rate Upper 95% Confidence Interval',
                      'Crude Rate Standard Error']
    
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Convert percentage column
    if '% of Total Deaths' in df.columns:
        df['% of Total Deaths'] = df['% of Total Deaths'].str.rstrip('%').astype(float) / 100
    
    return df

In [3]:
df = read_cdc_wonder('cod-male-5yr.txt')
print(f"Loaded {len(df)} rows of data")
print("\nFirst few rows:")
print(df.head())

print("\nData types of columns:")
print(df.dtypes)

Data ends on line 26080 out of 26162 lines
Loaded 26079 rows of data

First few rows:
   Notes        Cause of death Cause of death Code Five-Year Age Groups  \
0    NaN  Salmonella enteritis               A02.0          55-59 years   
1    NaN  Salmonella enteritis               A02.0         60-64 years    
2    NaN  Salmonella enteritis               A02.0          65-69 years   
3    NaN  Salmonella enteritis               A02.0          70-74 years   
4    NaN  Salmonella enteritis               A02.0          75-79 years   

  Five-Year Age Groups Code  Deaths   Population  Crude Rate  \
0                     55-59      13  201411886.0         NaN   
1                     60-64      10  168055475.0         NaN   
2                     65-69      11  133793292.0         NaN   
3                     70-74      15  102736281.0         NaN   
4                     75-79      20   75129413.0         0.0   

   Crude Rate Lower 95% Confidence Interval  \
0                              

In [4]:
df.head()

,Notes,Cause of death,Cause of death Code,Five-Year Age Groups,Five-Year Age Groups Code,Deaths,Population,Crude Rate,Crude Rate Lower 95% Confidence Interval,Crude Rate Upper 95% Confidence Interval,Crude Rate Standard Error,% of Total Deaths
0,NaN,Salmonella enteritis,A02.0,55-59 years,55-59,13,201411886.0,NaN,0.0,0.0,0.0,0.0
1,NaN,Salmonella enteritis,A02.0,60-64 years,60-64,10,168055475.0,NaN,0.0,0.0,0.0,0.0
2,NaN,Salmonella enteritis,A02.0,65-69 years,65-69,11,133793292.0,NaN,0.0,0.0,0.0,0.0
3,NaN,Salmonella enteritis,A02.0,70-74 years,70-74,15,102736281.0,NaN,0.0,0.0,0.0,0.0
4,NaN,Salmonella enteritis,A02.0,75-79 years,75-79,20,75129413.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df.groupby(['Cause of death', 'Cause of death Code'])['Deaths'].sum().sort_values()

Cause of death                                                Cause of death Code
Double outlet left ventricle                                  Q20.2                       10
Gastrojejunal ulcer, chronic or unspecified with perforation  K28.5                       10
Generalized anxiety disorder                                  F41.1                       10
Glans penis - Malignant neoplasms                             C60.1                       10
Ulcerative (chronic) enterocolitis                            K51.0                       10
                                                                                      ...   
Atherosclerotic cardiovascular disease, so described          I25.0                   804971
Chronic obstructive pulmonary disease, unspecified            J44.9                  1098360
Acute myocardial infarction, unspecified                      I21.9                  1629859
Bronchus or lung, unspecified - Malignant neoplasms           C34.9              